## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import patsy

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours, TomekLinks
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.pipeline import Pipeline

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('./chronic_kidney_disease_full.csv')

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

In [3]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [4]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

Answer:

I'd like to examine the relationship between someone's age, blood pressure, red blood cell count, sodium, potassium, white blood cell count, etc., and whether or not they exhibit symptoms of/have CKD. 

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [5]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [6]:
df_copy = df.dropna()

In [7]:
df_copy.shape

(158, 25)

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

Answer:

158 rows would remain.

One downside of dropping rows with missing data is that we would be losing the data in the columns that do have valid values.

Another downside of dropping rows is that we would no longer be dealing with a 'random' sample of respondents.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

Answer:

    1. Any columns/features that involve human measurement may contain human error, i.e. faulty practices.
    2. Some columns deal with data that can be largely temporary, making them largely influenced by timing.
    3. Some records may have been mistakenly adjusted/rounded/recorded incorrectly.

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

** Answer: **

Build a model that tells everyone that they have CKD; the only error that could occur is a Type I Error, or a False Positive.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

** Answer: **

We optimized ** Sensitivity ** and the **False Negative Rate** by minimizing ** False Negatives**.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

** Answer: **

One disadvantage to the model described in problem 6 which tells everyone that they have CKD is that the model will produce many False Positives, causing a lot of undue stress and worry.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple) model can I create that will accomplish this?

** Answer: **

Build a model that tells everyone that they do not have CKD; the only error that could occur is a Type II Error, or a False Negative.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

** Answer: **

We optimized ** Specificity ** and the **False Positive Rate** by minimizing ** False Positives**.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

** Answer: **

One disadvantage to the model described in problem 9 which tells everyone that they do not have CKD is that the model will produce many False Negatives, causing a lot of sick people to believe they are not sick and not take any action to help themselves.

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [8]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
rbc      248 non-null object
pc       335 non-null object
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
sod      313 non-null float64
pot      312 non-null float64
hemo     348 non-null float64
pcv      329 non-null float64
wbcc     294 non-null float64
rbcc     269 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(14), object(11)
memory usage: 78.2+ KB


In [10]:
df.drop(['rbc','pc','sod','pot','pcv','wbcc','rbcc'], axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
age      391 non-null float64
bp       388 non-null float64
sg       353 non-null float64
al       354 non-null float64
su       351 non-null float64
pcc      396 non-null object
ba       396 non-null object
bgr      356 non-null float64
bu       381 non-null float64
sc       383 non-null float64
hemo     348 non-null float64
htn      398 non-null object
dm       398 non-null object
cad      398 non-null object
appet    399 non-null object
pe       399 non-null object
ane      399 non-null object
class    400 non-null object
dtypes: float64(9), object(9)
memory usage: 56.3+ KB


---

## Build model:

In [12]:
df_v1 = df

In [13]:
df_v1.replace(np.nan, 0, inplace=True)

In [14]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
age      400 non-null float64
bp       400 non-null float64
sg       400 non-null float64
al       400 non-null float64
su       400 non-null float64
pcc      400 non-null object
ba       400 non-null object
bgr      400 non-null float64
bu       400 non-null float64
sc       400 non-null float64
hemo     400 non-null float64
htn      400 non-null object
dm       400 non-null object
cad      400 non-null object
appet    400 non-null object
pe       400 non-null object
ane      400 non-null object
class    400 non-null object
dtypes: float64(9), object(9)
memory usage: 56.3+ KB


---

In [15]:
df_v1['pcc'].value_counts()

notpresent    354
present        42
0               4
Name: pcc, dtype: int64

In [16]:
df_v1['pcc'].replace(0, 'present', inplace=True)

In [17]:
df_v1['pcc'].value_counts()

notpresent    354
present        46
Name: pcc, dtype: int64

---

In [18]:
df_v1['ba'].value_counts()

notpresent    374
present        22
0               4
Name: ba, dtype: int64

In [19]:
df_v1['ba'].replace(0, 'present', inplace=True)

In [20]:
df_v1['ba'].value_counts()

notpresent    374
present        26
Name: ba, dtype: int64

---

In [21]:
df_v1['htn'].value_counts()

no     251
yes    147
0        2
Name: htn, dtype: int64

In [22]:
df_v1['htn'].replace(0, 'yes', inplace=True)

In [23]:
df_v1['htn'].value_counts()

no     251
yes    149
Name: htn, dtype: int64

---

In [24]:
df_v1['dm'].value_counts()

no     261
yes    137
0        2
Name: dm, dtype: int64

In [25]:
df_v1['dm'].replace(0, 'yes', inplace=True)

In [26]:
df_v1['dm'].value_counts()

no     261
yes    139
Name: dm, dtype: int64

---

In [27]:
df_v1['cad'].value_counts()

no     364
yes     34
0        2
Name: cad, dtype: int64

In [28]:
df_v1['cad'].replace(0, 'yes', inplace=True)

In [29]:
df_v1['cad'].value_counts()

no     364
yes     36
Name: cad, dtype: int64

---

In [30]:
df_v1['appet'].value_counts()

good    317
poor     82
0         1
Name: appet, dtype: int64

In [31]:
df_v1['appet'].replace(0, 'poor', inplace=True)

In [32]:
df_v1['appet'].value_counts()

good    317
poor     83
Name: appet, dtype: int64

---

In [33]:
df_v1['pe'].value_counts()

no     323
yes     76
0        1
Name: pe, dtype: int64

In [34]:
df_v1['pe'].replace(0, 'yes', inplace=True)

In [35]:
df_v1['pe'].value_counts()

no     323
yes     77
Name: pe, dtype: int64

---

In [36]:
df_v1['ane'].value_counts()

no     339
yes     60
0        1
Name: ane, dtype: int64

In [37]:
df_v1['ane'].replace(0, 'yes', inplace=True)

In [38]:
df_v1['ane'].value_counts()

no     339
yes     61
Name: ane, dtype: int64

---

In [39]:
df_v1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

---

In [40]:
df_v1 = pd.get_dummies(data=df_v1, drop_first=True)

In [41]:
df_v1.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class_notckd'],
      dtype='object')

In [42]:
df_v1.drop('class_notckd', axis=1, inplace=True)

In [43]:
df_v1 ['class'] = df['class']

In [44]:
df_v1.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class'],
      dtype='object')

In [45]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
age            400 non-null float64
bp             400 non-null float64
sg             400 non-null float64
al             400 non-null float64
su             400 non-null float64
bgr            400 non-null float64
bu             400 non-null float64
sc             400 non-null float64
hemo           400 non-null float64
pcc_present    400 non-null uint8
ba_present     400 non-null uint8
htn_yes        400 non-null uint8
dm_yes         400 non-null uint8
cad_yes        400 non-null uint8
appet_poor     400 non-null uint8
pe_yes         400 non-null uint8
ane_yes        400 non-null uint8
class          400 non-null object
dtypes: float64(9), object(1), uint8(8)
memory usage: 34.5+ KB


In [46]:
v1col_list = list(df_v1.columns)

In [47]:
v1_features = []

In [48]:
[v1_features.append(col) for col in v1col_list if col != 'class']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [49]:
v1_features

['age',
 'bp',
 'sg',
 'al',
 'su',
 'bgr',
 'bu',
 'sc',
 'hemo',
 'pcc_present',
 'ba_present',
 'htn_yes',
 'dm_yes',
 'cad_yes',
 'appet_poor',
 'pe_yes',
 'ane_yes']

In [50]:
X = df_v1[v1_features]
y = df_v1['class']

In [51]:
poly = PolynomialFeatures(include_bias=False, degree=2)

In [52]:
X_poly = poly.fit_transform(X)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state = 42)

In [54]:
ss = StandardScaler()

In [55]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [56]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [57]:
logreg = LogisticRegression()

In [58]:
logreg.fit(X_train_sc, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [59]:
logreg.score(X_train_sc, y_train)

0.9666666666666667

In [60]:
logreg.score(X_test_sc, y_test)

0.96

---

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

In [61]:
logreg.coef_

array([[ 5.40851087e-01,  5.58109976e-01,  2.19941105e-01,
        -6.12890223e-01, -6.88409429e-02,  1.33080813e-01,
         3.61325401e-02, -4.52480105e-01, -5.37444596e-01,
         5.16838478e-03,  1.16224756e-01, -2.36455155e-01,
        -2.11777352e-01,  2.24632368e-02, -2.97126740e-01,
        -2.57762039e-01, -1.64064082e-01, -1.02086764e+00,
        -4.53898141e-01,  2.61890019e-01, -4.06790313e-01,
         8.07661475e-02, -4.24593458e-02,  2.88131408e-01,
        -1.75970997e-01,  1.01094450e+00, -8.43953715e-02,
        -1.37781531e-03, -1.00542429e-01, -1.89899390e-01,
        -5.44909994e-02, -4.57157278e-02, -9.24270016e-02,
        -3.35860995e-02, -6.75271979e-01,  6.95450384e-03,
        -3.69065816e-01, -1.27494381e-02,  2.00828300e-01,
         4.70786263e-01, -2.46425314e-01, -3.78196906e-01,
        -1.76327914e-02,  6.21969851e-02, -1.95142647e-01,
        -1.38361915e-01, -4.68452706e-03, -1.32912764e-01,
        -1.13391895e-01, -9.61926095e-02,  4.38050638e-0

In [62]:
np.exp(0.11018577)

1.1164854606988666

## Coefficient for Beta 1 Interpretation:

Beta 1's coefficient of 0.11018577 implies that a respondent is more and more likely to have CKD for every year he/she is older.

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

** Answer: **

I believe that's the case because it's ** simple **, and can ** predict probabilities. **

** Advantages of Logistic Regression: **
    1.  Logistic Regression will predict continuous values.
    2.  Logistic Regression connects regression and classification.
    3.  Logistic Regression can predict probabilities.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

### It makes sense to generate a confusion matrix on our test data because we do all of our testing on our test data; our test data is our unseen data.

In [63]:
predictions = logreg.predict(X_test_sc)

In [64]:
from sklearn.metrics import confusion_matrix

In [65]:
cm = confusion_matrix(y_test, predictions)

In [66]:
cm

array([[62,  3],
       [ 1, 34]])

In [67]:
cm = pd.DataFrame(cm, columns=['Predicted Negative','Predicted Positive'], index=['Actual Negative','Actual Positive'])

In [68]:
cm

,Predicted Negative,Predicted Positive
Actual Negative,62,3
Actual Positive,1,34


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

** Answer: **

In this hospital case, we want to optimize for ** sensitivity** or **recall**; although predicting someone is positive for CKD when they are actually negative, a False Negative/Type II Error, is a bad mistake to make, it is not as bad as predicting someone is negative for CKD when they are actually positve, a False Positive/Type I Error.

A False Negative/Type II Error can be remedied within a few months when the person being tested is screened again - the only harm down, while unnecessary and potentially considerable, would be emotional distress.

A False Positive/Type I Error, on the other hand, would result in a sick person going about their life as if they were healthy, not addressing their ailing condition, possibly leading to even more serious health concerns or even death.



### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

**Answer:**

Unbalanced classes are generally a problem because the minority class is at risk of not having enough exposure during the model process to be accounted for in the model.  In other words, since there is so little of the minority class(es), they model cannot sufficiently pick up its signal, resulting in its attributes being overlooked/missed during the prediction process.

I do not believe this is a problem in this particular CKD analysis because the minority class, those without CKD, account for 37.5% of the data set, not a small enough percentage to be concerned about unbalanced-class-issues.

I would respond by offering to build two versions of the model, one that does not make any changes to offset the unbalanced classes, and one that sets both classes equal by either: 1. **oversampling the minority**, or negative CKD cases; or 2. **undersampling the majority**, or positive CKD cases.  This would allow us to observe if there are any measurable differences in the models' results.

In [69]:
df['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [70]:
df_v1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 4,600 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with 5,000 observations, of which only about 150 are non-CKD individuals.

In [71]:
df_v1.shape

(400, 18)

In [72]:
df_v1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [73]:
df_v1_maj = df_v1[ df_v1['class'] == 'ckd' ]
df_v1_min = df_v1[ df_v1['class'] == 'notckd' ]

In [74]:
df_upsample = resample(df_v1_maj, replace = True, n_samples = 4850, random_state = 42)
df_upsample = pd.concat([df_upsample, df_v1_min])

In [75]:
df_upsample['class'].value_counts()

ckd       4850
notckd     150
Name: class, dtype: int64

### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?

In [76]:
X = df_upsample[v1_features]
y = df_upsample['class']

In [77]:
X_poly = poly.fit_transform(X)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, random_state = 42)

In [79]:
ss.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [80]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [81]:
logreg.fit(X_train_sc, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [82]:
# Earlier score was 0.96666666666

In [83]:
logreg.score(X_train_sc, y_train)

0.9962666666666666

In [84]:
# Earlier score was 0.96

In [85]:
logreg.score(X_test_sc, y_test)

0.9976

In [86]:
predictions = logreg.predict(X_test_sc)

In [87]:
cm = confusion_matrix(y_test, predictions)

In [88]:
cm

array([[1208,    0],
       [   3,   39]])

In [89]:
cm = pd.DataFrame(cm, columns=['Predicted Negative','Predicted Positive'], index=['Actual Negative','Actual Positive'])

In [90]:
cm

,Predicted Negative,Predicted Positive
Actual Negative,1208,0
Actual Positive,3,39


In [91]:
1208+3+39

1250

### The impact of unbalanced classes on logistic regression is that it seems to have considerably affected the specificity of the model, in this case, as there are 0 False Positives out of a total of 1,250 predictions.  Generally, unbalanced classes don't allow models enough exposure to the minority class to be able to recognize a minority instance in predictions.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!

---